In [1]:
using JWAS,JWAS.Datasets,DataFrames,CSV, LinearAlgebra, DelimitedFiles, Profile, ProfileView

In [2]:
#load input files from XSim
phenofile = readdlm("../Data-Sim\\Phenotypes-ssbr1.txt");
genofile = readdlm("../Data-Sim\\Genotypes-ssbr1.txt");
pedfile = readdlm("../Data-Sim\\Pedigree-ss.txt");



In [ ]:
# no spaces in final file formats

In [14]:
#remove spaces after 1st function
function ped_ssbr_convert_1(pedfile)
    ## pedfile input in only one format
    ped_Array = Array{String}(undef, 1601);
    ped_Array[1] = "ID,Sire,Dam";
    temp_arr = Array{String}(undef, 3)
    ## convert the pedigree array from Float64 to Int and append the header to the file
    ## For Pedigree file IDs, Sires and Dams must be string in format
    f = (x) -> Int(x);
    @time for i in 1:1600
        y = pedfile[i, :]
        y = f.(y)
        y = string(y)
        z = ""
        s = split(y, "")
        l = length(s)
        for j in 1:length(s)
            if j !== 1 && j !== length(s) && y[j] !== ' '
             z = z*y[j]
            end
        end
        # this converts "[1, 2, 3]" to "1, 2, 3"
        ped_Array[i + 1] = z
    end
    return ped_Array
end

ped_ssbr_convert_1 (generic function with 1 method)

In [15]:
ped_out_1 = ped_ssbr_convert_1(readdlm("../Data-Sim\\Pedigree-ss.txt"))

  0.032309 seconds (223.65 k allocations: 11.380 MiB)


1601-element Array{String,1}:
 "ID,Sire,Dam"   
 "1701,1041,1362"
 "1702,1141,1376"
 "1703,894,1463" 
 "1704,972,1265" 
 "1705,764,1295" 
 "1706,825,1290" 
 "1707,1108,1676"
 "1708,886,1626" 
 "1709,1168,1564"
 "1710,762,1348" 
 "1711,941,1385" 
 "1712,1102,1622"
 ⋮               
 "3289,1115,1265"
 "3290,1147,1666"
 "3291,1114,1567"
 "3292,782,1375" 
 "3293,866,1434" 
 "3294,1145,1448"
 "3295,740,1307" 
 "3296,1194,1399"
 "3297,820,1400" 
 "3298,1077,1655"
 "3299,995,1389" 
 "3300,897,1235" 

In [63]:
### first transform "[1, 2, 3]" to "1, 2, 3" 
### then transform "1, 2, 3" to "1,2,3"
x = "1, 2, 3";
x =  split(x, ' ');
#
x = "a" .* x
#x
# split(x[2], "")
## combine into "a1,a2,a3"
x[1]*x[2]*x[3]


"a[1,a2,a3]"

In [80]:

function ped_ssbr_append_char(ped_Array)
    #f1 = (x) -> x*"a";
    ## split the file into components
    new_Arr = Array{String}(undef, 1601);
        
    for j in 2:1601
        #temp = string(ped_Array[j])
        #println(temp)
        x = split(string(ped_Array[j]), ",")
        #println(length(x))
        x = "a" .* x
        #println(x)
        new_Arr[j] = x[1]*","*x[2]*","*x[3]
    end
    new_Arr[1] = "ID,sire,dam"
    return new_Arr
end

ped_ssbr_append_char (generic function with 1 method)

In [81]:
ped_ssbr_append_char(ped_copy)

1601-element Array{String,1}:
 "ID,sire,dam"      
 "a1701,a1041,a1362"
 "a1702,a1141,a1376"
 "a1703,a894,a1463" 
 "a1704,a972,a1265" 
 "a1705,a764,a1295" 
 "a1706,a825,a1290" 
 "a1707,a1108,a1676"
 "a1708,a886,a1626" 
 "a1709,a1168,a1564"
 "a1710,a762,a1348" 
 "a1711,a941,a1385" 
 "a1712,a1102,a1622"
 ⋮                  
 "a3289,a1115,a1265"
 "a3290,a1147,a1666"
 "a3291,a1114,a1567"
 "a3292,a782,a1375" 
 "a3293,a866,a1434" 
 "a3294,a1145,a1448"
 "a3295,a740,a1307" 
 "a3296,a1194,a1399"
 "a3297,a820,a1400" 
 "a3298,a1077,a1655"
 "a3299,a995,a1389" 
 "a3300,a897,a1235" 

In [ ]:
###################################################################################
## Do similar conversion for genotypes except keep the values from 2:end - {Float}
## create a copy of the phenofile
## PHENOTYPE FILE FORMATTING
function pheno_ssbr_format(phenofile)
    pheno_copy = phenofile;

    ## extract rowIDs and convert to Int 
    rowIDs = Array{Any}(undef, 1601);
    rowIDs[2:end] = pedfile[:, 1];
    rowIDs[1] = "ID";
    f = (x) -> Int(x);
    for i in 2:1601
        rowIDs[i] = f.(rowIDs[i])
    end
    
    
#     f2 = (x) -> string(x);
#     for i in 2:1601
#         rowIDs[i] = f2.(rowIDs[i])
#     end
    ## for Ids in phenotype, genotype and pedigree add a string "a"
    for i in 2:1601
        rowIDs[i] = "a"*string(rowIDs[i])
    end
    
    ## concatenate the rowIDs
    pheno_cat = hcat(rowIDs, pheno_copy)
    pheno_array = Array{String}(undef, 1601)
    pheno_array[1] = "ID, y"
    for j in 2:1601
        m1 = pheno_cat[j, 2];
        m2 = string(m1);
        
        pheno_array[j] = pheno_cat[j, 1]*","*m2 
    end
    return pheno_array
end

In [86]:
#################################################################################
## GENOTYPE FILE FORMATTING
## use the genotype file and do same transformation for genotypes
## genotype script 
function geno_ssbr_format(genofile)
    geno_copy = genofile;
    pedfile = readdlm("../Data-Sim\\Pedigree-ss.txt");

    ## Add rowIDs(hcat) and marker IDs(vcat)
    ## extract rowIDs and convert to Int and then to string 
    rowIDs = Array{Any}(undef, 1601);
    rowIDs[2:end] = pedfile[:, 1];
    rowIDs[1] = "ID";
    f = (x) -> Int(x);
    for i in 2:1601
        rowIDs[i] = f.(rowIDs[i])
    end

#     f2 = (x) -> string(x);
#     for i in 2:1601
#         rowIDs[i] = f2.(rowIDs[i])
#     end
    for i in 2:1601
        rowIDs[i] = "a"*string(rowIDs[i])
    end
    
    ## hcat rowIDs
    geno_cat1 = Array{Any}(1601, 4001);
    geno_cat1[2:1601, :] = hcat(rowIDs[2:1601], geno_copy)
    println("genotype file concatenated with ID's: ", "a1, 1, 0, 2")
    
    ## add markerIDs and vcat
    empty_l = Array{String}(undef, 4000)
    empty_l[1] = "ID"
    empty_l[2:4000] = repeat(["m"], 3999)

    for i in 2:4000
        empty_l[i] = empty_l[i] * "$(i-1)"
    end
    
    len = 3999
    println("$len markerIDs created")
    ## Header split for genotype file
    H_split = ""
    for j in 1:4000
        H_split = H_split*empty_l[j]        
        if j == 4000
            H_split = H_split
        else
            H_split = H_split*","
        end
    end
    println("markerIDs split into:", "ID, m1, m2, m3", H_split )
    ## return the H_split for 1st column of geno_formatted
    
    
    ## Main split for genotype file 
    ## Main Split
    ## convert the float to Int and then string split 
    f = x -> Int(x);
    geno_string = Array{String}(undef, 1601);
    @time for j in 2:1601
        v = geno_cat1[j, 2:end]
        #println(v)
        v = f.(v)
        v = string(v)
        v = split(v, "")
        len = length(v)
        z = ""
        #println(v)
        for i in 1:len
            if i !== 1 && i !== 2 && i !== 3 && i !== len
                z = z*v[i]
            end
        end
        #println(z)
#         geno_string[j] = string(rowIDs[j])
#         #println(geno_string[j])
#         geno_string[j] = geno_string[j]*","*z
#         #println(geno_string[j])
        geno_string[j] = rowIDs[j]*","*z
    end
    println("Matrix split into", "a1, 0, 1, 2")
    geno_string[1] = H_split
    return  geno_string, rowIDs
end

geno_ssbr_format (generic function with 1 method)

## Test the formatting functions on XSim data

In [82]:
#Testing pedigree output
ped_Array = ped_ssbr_convert_1(pedfile);
ped_out = ped_ssbr_append_char(ped_Array)

  0.031947 seconds (223.65 k allocations: 11.380 MiB, 24.32% gc time)


1601-element Array{String,1}:
 "ID,sire,dam"      
 "a1701,a1041,a1362"
 "a1702,a1141,a1376"
 "a1703,a894,a1463" 
 "a1704,a972,a1265" 
 "a1705,a764,a1295" 
 "a1706,a825,a1290" 
 "a1707,a1108,a1676"
 "a1708,a886,a1626" 
 "a1709,a1168,a1564"
 "a1710,a762,a1348" 
 "a1711,a941,a1385" 
 "a1712,a1102,a1622"
 ⋮                  
 "a3289,a1115,a1265"
 "a3290,a1147,a1666"
 "a3291,a1114,a1567"
 "a3292,a782,a1375" 
 "a3293,a866,a1434" 
 "a3294,a1145,a1448"
 "a3295,a740,a1307" 
 "a3296,a1194,a1399"
 "a3297,a820,a1400" 
 "a3298,a1077,a1655"
 "a3299,a995,a1389" 
 "a3300,a897,a1235" 

In [43]:
ped_copy = ped_Array

1601-element Array{String,1}:
 "ID,Sire,Dam"   
 "1701,1041,1362"
 "1702,1141,1376"
 "1703,894,1463" 
 "1704,972,1265" 
 "1705,764,1295" 
 "1706,825,1290" 
 "1707,1108,1676"
 "1708,886,1626" 
 "1709,1168,1564"
 "1710,762,1348" 
 "1711,941,1385" 
 "1712,1102,1622"
 ⋮               
 "3289,1115,1265"
 "3290,1147,1666"
 "3291,1114,1567"
 "3292,782,1375" 
 "3293,866,1434" 
 "3294,1145,1448"
 "3295,740,1307" 
 "3296,1194,1399"
 "3297,820,1400" 
 "3298,1077,1655"
 "3299,995,1389" 
 "3300,897,1235" 

In [ ]:
#Testing Phenotype output
pheno_out = pheno_ssbr_format(phenofile)

In [83]:
## write output of XSIM_converted pedigree file
f_ped = open("ped_XSim_out.txt", "w");
writedlm(f_ped, ped_out);
close(f_ped)

In [ ]:
## write output of XSim converted genotype file
f_pheno = open("pheno_XSim_out.txt", "w");
writedlm(f_pheno, pheno_out);
close(f_pheno)

In [ ]:
f_geno = open("geno_XSim_out.txt", "w");
writedlm(f_geno, geno_test1);
close(f_geno)

In [90]:
using JWAS,JWAS.Datasets,DataFrames,CSV, LinearAlgebra

phenofile  = "./pheno_XSim_out.txt" #Datasets.dataset("example","phenotypes_ssbr.txt")
pedfile    = "./ped_XSim_out.txt" #Datasets.dataset("example","pedigree.txt")
genofile   = "./geno_XSim_out.txt"#Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

Finished!


In [92]:
model_equation1  ="y = intercept";

R      = 1.0
model1 = build_model(model_equation1,R);

# set_covariate(model1,"x1");

# G1 = 1.0
# G2 = [1.0 0.5
#        0.5 1.0]
# set_random(model1,"x2",G1);
# set_random(model1,"ID dam",pedigree,G2);

G3 =1.0
add_genotypes(model1,genofile,G3,separator=',');

outputEBV(model1, ["a1701","a1702","a1703"]);
outputMCMCsamples(model1)
out2=runMCMC(model1,Pi=0.95,estimatePi=true,phenotypes,methods="BayesC",single_step_analysis=true,
    pedigree=pedigree,chain_length=50000,output_samples_frequency=100,outputEBV=true, missing_phenotypes=true);

keys(out2)

out2["EBV_y_2"]

out2["Posterior mean of marker effects"]


3999 markers on 1600 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.010166


A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
ϵ               factor       random             964
J               covariate    fixed                1

MCMC Information:

methods                                      BayesC
chain_length                                  50000
burnin                                            0
estimatePi                                     true
starting_value                                false
printout_frequency                            50001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                             true
update_priors_

running MCMC for BayesC...100%|█████████████████████████| Time: 0:07:04


KeyError: KeyError: key "EBV_y_2" not found

In [89]:
?runMCMC

search: runMCMC



```
runMCMC(mme,df;Pi=0.0,estimatePi=false,chain_length=1000,burnin = 0,starting_value=false,printout_frequency=100,
missing_phenotypes=false,constraint=false,methods="conventional (no markers)",output_samples_frequency::Int64 = 0,
printout_model_info=true,outputEBV=false)
```

Run MCMC (marker information included or not) with sampling of variance components.

  * available **methods** include "conventional (no markers)", "RR-BLUP", "BayesB", "BayesC".
  * save MCMC samples every **output*samples*frequency** iterations to files **output_file** defaulting to `MCMC_samples`.
  * the first **burnin** iterations are discarded at the beginning of an MCMC run
  * **Pi** for single-trait analyses is a number; **Pi** for multi-trait analyses is a dictionary such as `Pi=Dict([1.0; 1.0]=>0.7,[1.0; 0.0]=>0.1,[0.0; 1.0]=>0.1,[0.0; 0.0]=>0.1)`,

      * if Pi (Π) is not provided in multi-trait analysis, it will be generated assuming all markers have effects on all traits.
  * **starting_value** can be provided as a vector for all location parameteres except marker effects.
  * print out the monte carlo mean in REPL with **printout_frequency**
  * **constraint**=true if constrain residual covariances between traits to be zeros.
  * Individual EBVs are returned if **outputEBV**=true.


In [ ]:
?readdlm